Importação bibliotecas

In [134]:
import pandas as pd # Fundamental para análise e manipulação de dados
import requests # Para fazer requisições HTTP (acessar APIs, baixar dados da web)
import os # Para interagir com o sistema operacional (manipular arquivos, pastas)
from datetime import datetime # Módulo para trabalhar com datas e horas
import yfinance as yt # Baixar dados do mercado financeiro (Yahoo Finance)
from pytrends.request import TrendReq # Extrair dados de tendências de busca (Google Trends)
from sidrapy import get_table # Função específica para buscar dados de tabelas do SIDRA (IBGE)

Variáveis centrais

In [135]:
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y/%m/%d')

data_raw_path = '../data_raw/'
os.makedirs(data_raw_path, exist_ok=True)

Definição {dicionário} de indicadores BCB

In [136]:
indicadores_bcb = {
    '432': 'selic_meta',      # SELIC (Meta) - Mensal
    '433': 'ipca_mensal',     # IPCA (Var. % mensal) - Mensal
    '24364': 'ibc_br',        # IBC-Br (Proxy do PIB, c/ ajuste) - Mensal
    '4393': 'icc_consumidor', # Índice de Confiança do Consumidor - Mensal
    '24364': 'pib'            # Produto interno bruto - Mensal
}

Função de Coleta do Banco Central (SGS)

- Busca os indicadores e salva em um arquivo CSV na pasta data_raw

In [137]:
def dados_bc (codigo_indicador, nome_indicador):

    start_date_fmt = datetime.strptime(start_date, '%Y-%m-%d').strftime('%d/%m/%Y')
    end_date_fmt = datetime.now().strftime('%d/%m/%Y')

    url_api = f"http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_indicador}/dados?formato=json&dataInicial={start_date_fmt}&dataFinal={end_date_fmt}"
    
    #Chamar a API
    response = requests.get(url_api)
    
    if response.status_code == 200: #Código 200 significa êxito
        dados_json = response.json()
        
        if not dados_json: # Check se a API retornar sem dados
            print(f"   > AVISO: Série {nome_indicador} não retornou dados.")
            return

        # Converte para DataFrame
        df = pd.DataFrame(dados_json)
        
        # Renomeia a coluna 'valor' para o nome do indicador
        df = df.rename(columns={'valor': nome_indicador})
        
        # Salvar o arquivo CSV individual (Criando novas pastas)
        file_path = os.path.join(data_raw_path, f'{nome_indicador}.csv')
        df.to_csv(file_path, index=False)
        
        print(f"   > Sucesso. Salvo em: {file_path}")
        
    else:
        print(f"   > ERRO! Falha ao buscar {nome_indicador}. Status: {response.status_code}")

Loop para a função de coleta BCB

In [138]:
for codigo, nome in indicadores_bcb.items():
    dados_bc(codigo, nome)

   > Sucesso. Salvo em: ../data_raw/selic_meta.csv
   > Sucesso. Salvo em: ../data_raw/ipca_mensal.csv
   > Sucesso. Salvo em: ../data_raw/pib.csv
   > Sucesso. Salvo em: ../data_raw/icc_consumidor.csv


Coleta Taxa de Desocupação (IBGE/SIDRA)

In [139]:
try:
    
    desemprego_df = get_table(
        table_code='4095',
        territorial_level='1',
        ibge_territorial_code='all',
        variable='4099',
        period='all' 
    )
    
    if not desemprego_df.empty:
        desemprego_df = desemprego_df.loc[1:, ['V', 'D2C']]
        desemprego_df.columns = ['taxa_desocupacao', 'periodo_codigo_trimestre']
        desemprego_df['taxa_desocupacao'] = pd.to_numeric(desemprego_df['taxa_desocupacao'], errors='coerce')
        # Filtro de data
        desemprego_df = desemprego_df[desemprego_df['periodo_codigo_trimestre'] >= '202001']
        
        file_path = os.path.join(data_raw_path, 'desemprego_trimestral.csv')
        desemprego_df.to_csv(file_path, index=False)
        print(f"   > Sucesso. Salvo em: {file_path}")
    else:
        print("   > AVISO: Dados de desemprego não retornaram.")
        
except Exception as e:
    print(f"   > ERRO GERAL! Falha ao buscar Taxa de Desocupação : {e}")

   > Sucesso. Salvo em: ../data_raw/desemprego_trimestral.csv
